In [197]:
import pybaseball
from pybaseball import statcast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score 
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import timezone
from bs4 import BeautifulSoup
import io
import requests
pybaseball.cache.enable()

In [198]:
url = 'https://docs.google.com/spreadsheets/d/1JgczhD5VDQ1EiXqVG-blttZcVwbZd5_Ne_mefUGwJnk/pub?output=csv'
res = requests.get(url)
ID = pd.read_csv(io.BytesIO(res.content), sep=',')
ID.dropna(subset=['MLBID'], inplace=True)
ID['MLBID'] = ID['MLBID'].astype(int)
BID = pd.read_csv("mlb-player-stats-Batters.csv")
PID = pd.read_csv("mlb-player-stats-P.csv")

In [199]:
def convert_name(name):
    if name == 'Rockies':
        return 'COL'
    elif name == 'Reds':
        return 'CIN'
    elif name == 'Mariners':
        return 'SEA'
    elif name == 'Nationals':
        return 'WAS'
    elif name == 'Yankees':
        return 'NYY'
    elif name == 'Astros':
        return 'HOU'
    elif name == 'Red Sox':
        return 'BOS'
    elif name == 'Athletics':
        return 'OAK'
    elif name == 'Mets':
        return 'NYM'
    elif name == 'Braves':
        return 'ATL'
    elif name == 'Giants':
        return 'SF'
    elif name == 'Brewers':
        return 'MIL'
    elif name == 'Rays':
        return 'TB'
    elif name == 'Royals':
        return 'KC'
    elif name == 'White Sox':
        return 'CWS'
    elif name == 'Cubs':
        return 'CHC'
    elif name == 'Angels':
        return 'LAA'
    elif name == 'Tigers':
        return 'DET'
    elif name == 'Diamondbacks':
        return 'ARI'
    elif name == 'Guardians':
        return 'CLE'
    elif name == 'Orioles':
        return 'BAL'
    elif name == 'Twins':
        return 'MIN'
    elif name == 'Marlins':
        return 'MIA'
    elif name == 'Phillies':
        return 'PHI'
    elif name == 'Rangers':
        return 'TEX'
    elif name == 'Dodgers':
        return 'LAD'
    elif name == 'Padres':
        return 'SD'
    elif name == 'Pirates':
        return 'PIT'
    elif name == 'Blue Jays':
        return 'TOR'
    elif name == 'Cardinals':
        return 'STL'
    else:
        return np.nan
    
def flip_names(name):
    first_name, last_name = name.split(", ")
    return f"{last_name} {first_name}"

In [200]:
def getDKData2024():
    eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
    todaysdate = eastern_time.strftime("%m-%d-%Y")
    url = 'https://rotogrinders.com/lineups/mlb?site=draftkings'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    gamelist = []
    gamecards = soup.findAll("div", {"class": "game-card-teams"})
    for x in gamecards:
        twoteams = x.findAll("span", {"class": "team-nameplate-mascot"})
        roadteam = convert_name(twoteams[0].text)
        hometeam = convert_name(twoteams[1].text)
        gamekey = "{}@{}".format(roadteam,hometeam)
        gamelist.append(gamekey)

    matchupsdf = pd.DataFrame()
    for game in gamelist:
        roadteam = game.split("@")[0]
        hometeam = game.split("@")[1]
        thisdf1 = pd.DataFrame({"Team": roadteam, "Opp": hometeam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        thisdf2 = pd.DataFrame({"Team": hometeam, "Opp": roadteam, "RoadTeam": roadteam, "HomeTeam": hometeam},index=[0])
        matchupsdf = pd.concat([matchupsdf,thisdf1,thisdf2])
        
    oppdict = dict(zip(matchupsdf.Team,matchupsdf.Opp))
    hometeamdict = dict(zip(matchupsdf.Team,matchupsdf.HomeTeam))
    roadteamdict = dict(zip(matchupsdf.Team,matchupsdf.RoadTeam))

    disabled_span_list = []
    for span in soup.findAll("span", {"class": "player-nameplate disabled"}):
        for a in span.findAll("a"):
            disabled_span_list.append(a.text)

    spdata = pd.DataFrame()
    for div in soup.findAll("span", {"class": "player-nameplate", "data-position": "SP"}):
        if "TBD" in str(div):
            playername = "TBD"
            pos = "SP"
            sal = 0
        else:
            for a in div.findAll('a', {'class': 'player-nameplate-name'}):
                playername = a.text.strip()

            strdiv = str(div)
            pos = strdiv[strdiv.find("data-position")+15:strdiv.find("data-salary")-2]
            sal = strdiv[strdiv.find("data-salary")+13:strdiv.find("<div class = 'player-nameplate-info'>")-3]
        try:
            ownership = strdiv[strdiv.find('<span class="small muted" data-auth="502">') + 42:strdiv.find('%')]
            ownership = ownership.replace("</span>", "")
            ownership = ownership.replace("</span", "")
            ownership = ownership.replace("</div>", "")
            ownership = ownership.replace(" ", "")
        except:
            ownership = np.nan

        thisspdata = pd.DataFrame([[playername, sal, ownership]], columns = ["Player", "Salary", "Ownership"])
        spdata = pd.concat([spdata, thisspdata])

    spdata2 = pd.merge(spdata, PID[["Player", "Team"]], left_on = ["Player"], right_on = ["Player"], how = "left").rename(columns = {"Team": "PitcherTeam"})
    spdata3 = pd.merge(spdata2, matchupsdf[["Team", "Opp"]], left_on = ["PitcherTeam"], right_on = ["Team"], how = "left").drop(columns = ["Team"])

    opp_spname_dict = dict(zip(spdata3.Opp, spdata3.Player))
    opp_spsal_dict = dict(zip(spdata3.Opp, spdata.Salary))
    opp_spown_dict = dict(zip(spdata3.Opp, spdata3.Ownership))

    ludf = pd.DataFrame()
    
    for li in soup.findAll("li", {"class": "lineup-card-player"}):
        for a in li.findAll("a", {"class": ["player-nameplate-name", "player-nameplate disabled"]}):
            playername = a.text

        listring = str(li)
        for span in li.find("span", {"class": "small"}):
            luspot = span.text
            luspot = luspot.replace("\n", "")
            luspot = luspot.strip()
            luspot = int(luspot)
        pos = listring[listring.find("data-position")+15:listring.find("data-salary")-2]
        sal = listring[listring.find("data-salary")+13:listring.find("<span class='small'>")-3]
        ownership = ownership.replace("</span>", "")
        ownership = ownership.replace("</span", "")
        ownership = ownership.replace("</li", "")
        ownership = ownership.replace("</div>", "")
        ownership = ownership.replace(" ", "")

        try:
            sal = int(sal)
        except:
            sal = 0
        thisludf = pd.DataFrame([[playername, luspot, sal, ownership]], columns = ["Player", "Spot", "Sal", "Ownership"])
        ludf = pd.concat([ludf, thisludf])

    ludf2 = pd.merge(ludf, BID[["Player", "Team"]], left_on = ["Player"], right_on = ["Player"], how = "left").rename(columns = {"Team": "BatterTeam"})
    ludf2['BatterTeam'] = ludf2['BatterTeam'].fillna(method='ffill')

    ludf2_teamlist = list(ludf2["BatterTeam"])

    dhteams = []
    for x in ludf2_teamlist:
        if ludf2_teamlist.count(x) > 11:
            if x in dhteams:
                pass
            else:
                dhteams.append(x)

    extract_dh = ludf2[ludf2["BatterTeam"].isin(dhteams)]
    new_ludf2 = ludf2[~ludf2["BatterTeam"].isin(dhteams)]

    new_team_list = []
    runcounter = 0

    for x in list(extract_dh["BatterTeam"].astype(str)):
        if runcounter < 18:
            new_team_list.append(x)
            runcounter += 1
        else:
            new_team_list.append(x+"2")
            runcounter += 1

    extract_dh["BatterTeam"] = new_team_list

    ludf2 = pd.concat([extract_dh, new_ludf2])
    ludf2["Opp"] = ludf2["BatterTeam"].map(oppdict)
    ludf2["HomeTeam"] = ludf2["BatterTeam"].map(hometeamdict)
    ludf2["RoadTeam"] = ludf2["BatterTeam"].map(roadteamdict)
    ludf2['SP'] = ludf2['BatterTeam'].map(opp_spname_dict)
    ludf2['SPSal'] = ludf2['BatterTeam'].map(opp_spsal_dict)
    ludf2['SPOwnership'] = ludf2['BatterTeam'].map(opp_spown_dict)
    ludf2['Date'] = todaysdate
    ludf2['Time'] = np.nan

    ludf3 = ludf2[['BatterTeam','RoadTeam','HomeTeam','Time','Spot','Player','Sal','Ownership','Date', "SP"]]

    dkdata = ludf3.copy()

    try:
        checknan = dkdata[["BatterTeam", "SP"]]
        getnans = checknan[["SP"].isna()]
        if len(getnans) == 0:
            nonans = 1
            nanmapdict = {}
        else:
            nonans = 0
            getnans["SP"] = disabled_span_list
            nanmapdict = dict(zip(getnans.Team, getnans.SP))
    except:
        pass

    try:
        dkdata["SP"] = np.where(dkdata["SP"].isna(), dkdata["BatterTeam"].map(nanmapdict), dkdata["SP"])
    except:
        pass
    
    for i in range(1, len(dkdata) - 1):
        if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i-1, 'BatterTeam']:
            if dkdata.loc[i, 'BatterTeam'] != dkdata.loc[i+1, 'BatterTeam']:
                dkdata.loc[i, 'BatterTeam'] = np.nan
                dkdata.loc[i, 'HomeTeam'] = np.nan
                dkdata.loc[i, 'RoadTeam'] = np.nan

    
    dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]] = dkdata[["BatterTeam", "RoadTeam", "HomeTeam"]].fillna(method='ffill')
    dkdata = dkdata.drop_duplicates(subset = ["Spot", "Player", "SP"], keep = "first")
    dkdata = dkdata.drop(columns = ["Time", "Sal", "Ownership"])

    dkdata['Date'] = pd.to_datetime(dkdata['Date'])
    dkdata['Date'] = dkdata['Date'].dt.strftime('%Y-%m-%d')
    dkdata = dkdata.set_index("Date")

    return(dkdata)

In [201]:
TodaysData = getDKData2024()

In [202]:
savant2023 = pd.read_csv("~/Desktop/Random-Projects/MLB/savant2023.csv")

In [203]:
eastern_time = datetime.datetime.now(timezone.utc).astimezone(timezone(datetime.timedelta(hours=-5)))
savant2024 = statcast(start_dt = "2024-03-28", end_dt = eastern_time.strftime("%Y-%m-%d"))

This is a large query, it may take a moment to complete


100%|██████████| 39/39 [00:15<00:00,  2.50it/s]


In [204]:
savant2024 = pd.merge(savant2024, ID[["MLBID", "MLBNAME"]], left_on = 'batter', right_on = 'MLBID', how = 'left')
savant2024.dropna(subset=['MLBNAME'], inplace=True)
savant2024 = savant2024.drop_duplicates(subset = ["pitch_type", "game_date", "release_speed", "release_pos_x", "release_pos_z", "player_name"], keep='first')

In [205]:
combined1 = pd.concat([savant2023, savant2024])
combined1 = combined1[combined1['inning'] == 1]
combined1['game_date'] = pd.to_datetime(combined1['game_date'])
combined1['game_date'] = combined1['game_date'].dt.strftime('%Y-%m-%d')
combined1["BatterTeam"] = np.nan
combined1['BatterTeam'] = np.where(combined1['inning_topbot'] == 'Top', combined1['away_team'], combined1['home_team'])
combined1["player_name"] = combined1["player_name"].apply(flip_names)

In [206]:
combined2 = combined1[["game_date", "home_team", "away_team", "inning_topbot", "stand", "at_bat_number", "pitch_number", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "on_1b", "on_2b", "on_3b", "events", "description", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "player_name", "delta_home_win_exp", "delta_run_exp", "away_score", "home_score"]].sort_values(by = ["game_date", "home_team", "away_team", "inning_topbot", "at_bat_number", "pitch_number"], ascending=[True, True, True, False, True, True])
combined2 = combined2.set_index("game_date").sort_index(ascending = True)

In [207]:
def fill_na_with_0_and_1(column):
    column_filled = column.fillna(0)
    column_filled[column_filled != 0] = 1
    return column_filled

combined2['on_1b'] = fill_na_with_0_and_1(combined2['on_1b'])
combined2['on_2b'] = fill_na_with_0_and_1(combined2['on_2b'])
combined2['on_3b'] = fill_na_with_0_and_1(combined2['on_3b'])

In [208]:
combined2['NRFI'] = ((combined2['away_score'] == 0) | (combined2['home_score'] == 0)).astype(int)

In [209]:
# At the play by play level
pbp = combined2[["home_team", "away_team", "stand", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "on_1b", "on_2b", "on_3b", "description", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "player_name", "delta_home_win_exp", "delta_run_exp", "NRFI"]].fillna(0)
pbp

,home_team,away_team,stand,BatterTeam,MLBNAME,balls,strikes,outs_when_up,on_1b,on_2b,...,launch_speed,launch_angle,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,p_throws,player_name,delta_home_win_exp,delta_run_exp,NRFI
game_date,,,,,,,,,,,,,,,,,,,,,
2023-03-30,BOS,BAL,L,BAL,Cedric Mullins,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,R,Corey Kluber,0.0,0.036,1
2023-03-30,BOS,BAL,L,BAL,Cedric Mullins,1,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,R,Corey Kluber,0.0,-0.046,1
2023-03-30,BOS,BAL,L,BAL,Cedric Mullins,1,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,R,Corey Kluber,0.0,-0.055,1
2023-03-30,BOS,BAL,L,BAL,Cedric Mullins,1,2,0,0.0,0.0,...,102.7,16.0,0.0,0.0,0.0,R,Corey Kluber,0.0,0.0,1
2023-03-30,BOS,BAL,L,BAL,Cedric Mullins,1,2,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,R,Corey Kluber,0.022,-0.173,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-03,WSH,TOR,R,WSH,Nick Senzel,1,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,L,Yusei Kikuchi,0.0,0.026,1
2024-05-03,WSH,TOR,R,WSH,Nick Senzel,2,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,L,Yusei Kikuchi,0.0,-0.035,1
2024-05-03,WSH,TOR,R,WSH,Nick Senzel,2,2,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,L,Yusei Kikuchi,0.0,0.04,1


In [210]:
# At an AB level
ab = combined2[["home_team", "away_team", "stand", "BatterTeam", "MLBNAME", "balls", "strikes", "outs_when_up", "on_1b", "on_2b", "on_3b", "events", "description", "hit_distance_sc", "launch_speed", "launch_angle", "estimated_ba_using_speedangle", "estimated_woba_using_speedangle", "woba_value", "p_throws", "player_name", "delta_home_win_exp", "delta_run_exp", "NRFI"]]
ab = ab.dropna(subset=["events"]).fillna(0)
ab

,home_team,away_team,stand,BatterTeam,MLBNAME,balls,strikes,outs_when_up,on_1b,on_2b,...,launch_speed,launch_angle,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,p_throws,player_name,delta_home_win_exp,delta_run_exp,NRFI
game_date,,,,,,,,,,,,,,,,,,,,,
2023-03-30,BOS,BAL,L,BAL,Cedric Mullins,1,2,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,R,Corey Kluber,0.022,-0.173,1
2023-03-30,BOS,BAL,L,BAL,Adley Rutschman,2,0,1,0.0,0.0,...,104.5,26.0,0.943,1.848,2.0,R,Corey Kluber,-0.103,0.933,1
2023-03-30,BOS,BAL,L,BAL,Anthony Santander,3,2,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,R,Corey Kluber,0.014,-0.207,1
2023-03-30,BOS,BAL,R,BAL,Ryan Mountcastle,3,2,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.7,R,Corey Kluber,-0.011,0.11,1
2023-03-30,BOS,BAL,R,BAL,Ramon Urias,0,1,2,1.0,1.0,...,60.5,-42.0,0.137,0.123,0.0,R,Corey Kluber,0.038,-0.408,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-03,WSH,TOR,R,TOR,Justin Turner,1,2,2,0.0,0.0,...,68.0,19.0,0.717,0.645,0.0,L,Patrick Corbin,0.036,-0.22,1
2024-05-03,WSH,TOR,R,WSH,Jacob Young,2,2,0,0.0,0.0,...,100.3,6.0,0.597,0.562,0.0,L,Yusei Kikuchi,-0.021,-0.205,1
2024-05-03,WSH,TOR,L,WSH,CJ Abrams,1,0,1,0.0,0.0,...,104.0,21.0,0.697,0.984,0.0,L,Yusei Kikuchi,-0.016,-0.185,1


In [211]:
# Encodes the values so the ML algos can read in everything
non_numeric_columns = pbp.select_dtypes(exclude=['float64', 'int64']).columns
label_encoder = LabelEncoder()
for col in non_numeric_columns:
    pbp[col] = label_encoder.fit_transform(pbp[col])

non_numeric_columns = ab.select_dtypes(exclude=['float64', 'int64']).columns
label_encoder = LabelEncoder()
for col in non_numeric_columns:
    ab[col] = label_encoder.fit_transform(ab[col])

non_numeric_columns = TodaysData.select_dtypes(exclude=['float64', 'int64']).columns
label_encoder = LabelEncoder()
for col in non_numeric_columns:
    TodaysData[col] = label_encoder.fit_transform(TodaysData[col])

In [212]:
pbp = pbp[["home_team", "away_team", "BatterTeam", "MLBNAME", "player_name", "NRFI"]]
ab = ab[["home_team", "away_team", "BatterTeam", "MLBNAME", "player_name", "NRFI"]]
TodaysData = TodaysData.rename(columns={'Player': 'MLBNAME', 'RoadTeam': 'away_team', "HomeTeam": "home_team", "SP": "player_name"}).drop(columns = ["Spot"])

pbp

,home_team,away_team,BatterTeam,MLBNAME,player_name,NRFI
game_date,,,,,,
2023-03-30,3,2,2,81,93,1
2023-03-30,3,2,2,81,93,1
2023-03-30,3,2,2,81,93,1
2023-03-30,3,2,2,81,93,1
2023-03-30,3,2,2,81,93,1
...,...,...,...,...,...,...
2024-05-03,29,28,29,364,419,1
2024-05-03,29,28,29,364,419,1
2024-05-03,29,28,29,364,419,1


In [230]:
pbpFeatures = ab.drop(columns = ["NRFI"]).values.reshape(-1, 5)
pbpLabel = ab["NRFI"].values.reshape(-1, 1)
TodayFeatures = TodaysData.values.reshape(-1, 5)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(pbpFeatures, pbpLabel)

# Make predictions on the testing set
RFpred = rf_classifier.predict(TodayFeatures)

#rmse = np.sqrt(mean_squared_error(pbpLabel, RFpred))
#print("Root Mean Squared Error:", round(rmse, 2))

# Calculate accuracy
#accuracy = accuracy_score(pbpLabel, RFpred)
#precision = precision_score(pbpLabel, RFpred)
#print("Accuracy:", round(accuracy, 4))
#print("Precision:", round(precision, 4))

TodaysData["Pred"] = RFpred
TodaysData

/Users/cbkaplinger/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,BatterTeam,away_team,home_team,MLBNAME,player_name,Pred
Date,,,,,,
2024-05-05,7,3,11,80,0,1
2024-05-05,7,3,11,36,0,1
2024-05-05,7,3,11,226,0,1
2024-05-05,7,3,11,71,0,1
2024-05-05,7,3,11,70,0,1
...,...,...,...,...,...,...
2024-05-05,20,12,10,30,14,1
2024-05-05,20,12,10,194,14,1
2024-05-05,20,12,10,43,14,1


In [214]:
x_train, x_test, y_train, y_test = train_test_split(ab.drop(columns=['NRFI']).values.reshape(-1, 5), ab["NRFI"].values.reshape(-1, 1), test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train, y_train)

# Make predictions on the testing set
RFpred = rf_classifier.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, RFpred))
print("Root Mean Squared Error:", round(rmse, 2))

# Calculate accuracy
accuracy = accuracy_score(y_test, RFpred)
precision = precision_score(y_test, RFpred)
print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))

/Users/cbkaplinger/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Root Mean Squared Error: 0.16
Accuracy: 0.9749
Precision: 0.9774


In [215]:
gb_classifier = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, random_state=42)
gb_classifier.fit(x_train, y_train)

# Make predictions on the testing set
GBpred = gb_classifier.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, GBpred))
print("Root Mean Squared Error:", round(rmse, 2))

# Calculate accuracy
accuracy = accuracy_score(y_test, GBpred)
precision = precision_score(y_test, GBpred)
print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))

/Users/cbkaplinger/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Root Mean Squared Error: 0.16
Accuracy: 0.9735
Precision: 0.9744


In [216]:
# Create KNN classifier
knn = KNeighborsClassifier(n_neighbors=100)

# Train the classifier
knn.fit(x_train, y_train)

# Make predictions on the testing set
Kpred = knn.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, Kpred))
print("Root Mean Squared Error:", round(rmse, 2))

# Calculate accuracy
accuracy = accuracy_score(y_test, Kpred)
precision = precision_score(y_test, Kpred)
print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))

Root Mean Squared Error: 0.16
Accuracy: 0.9737
Precision: 0.9737


/Users/cbkaplinger/Library/Python/3.9/lib/python/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [217]:
"""combined4 = combined3
combined4["RFpred"] = RFpred
combined4["GBpred"] = GBpred
combined4["Kpred"] = Kpred
combined4"""

'combined4 = combined3\ncombined4["RFpred"] = RFpred\ncombined4["GBpred"] = GBpred\ncombined4["Kpred"] = Kpred\ncombined4'

In [218]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(pbp)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test = x_test.astype(np.float32)
y_test = x_test.astype(np.float32)

# Define and compile the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units=50, activation='relu', input_shape=(x_train.shape[1], x_train.shape[2])),
    tf.keras.layers.Dense(units=1)
])
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
LTSM = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

# Make predictions
predictions = model.predict(x_test)

# Inverse scaling for predictions
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate root mean squared error
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("Root Mean Squared Error:", rmse)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
print("Accuracy:", round(accuracy, 4))
print("Precision:", round(precision, 4))

AttributeError: module 'tensorflow' has no attribute 'keras'